In [29]:
import pandas as pd
import os
import random

In [30]:
# Get paths to working directory and files folder
dir_path = os.path.dirname(os.path.abspath(os.curdir))
dir_path = os.path.join(dir_path, 'Mutational_Signatures')
print(dir_path)

c:\Files\github\Mutational_Signatures


In [31]:
# Read in the tables
cosmic_signatures_path = os.path.join(dir_path, 'cosmic_signatures')
arr = os.listdir(cosmic_signatures_path)
data_frames = []
data_frames_names = []
for file in arr:
    data_frames.append(pd.read_csv(os.path.join(cosmic_signatures_path, file)))
    data_frames_names.append(file.removesuffix('.csv'))

print(data_frames[1].to_string)
print(data_frames_names)

<bound method DataFrame.to_string of    Mutation  SBS4_GRCh37  SBS4_GRCh38  SBS4_mm9  SBS4_mm10  SBS4_rn6
0   A[C>A]A     0.042196     0.042451  0.047600   0.047599  0.047990
1   A[C>A]C     0.033297     0.032990  0.033653   0.033655  0.034018
2   A[C>A]G     0.015599     0.016116  0.014097   0.014094  0.016775
3   A[C>A]T     0.029498     0.029663  0.029589   0.029590  0.029143
4   A[C>G]A     0.006889     0.006931  0.007772   0.007771  0.007835
..      ...          ...          ...       ...        ...       ...
91  T[T>C]T     0.000636     0.000639  0.000544   0.000544  0.000536
92  T[T>G]A     0.000377     0.000372  0.000337   0.000337  0.000335
93  T[T>G]C     0.000174     0.000177  0.000178   0.000178  0.000179
94  T[T>G]G     0.002320     0.002339  0.002272   0.002272  0.002242
95  T[T>G]T     0.000568     0.000570  0.000485   0.000485  0.000479

[96 rows x 6 columns]>
['SBS18', 'SBS4', 'SBS6', 'SBS7a', 'SBS9']


In [32]:
df_grch37 = pd.DataFrame()
for i in range(len(data_frames_names)):
    column = data_frames[i][data_frames_names[i] + '_GRCh37']
    df_grch37 = pd.concat([df_grch37, column], axis = 1)
df_grch37

,SBS18_GRCh37,SBS4_GRCh37,SBS6_GRCh37,SBS7a_GRCh37,SBS9_GRCh37
0,0.051534,0.042196,0.000425,6.704351e-05,0.000558
1,0.015810,0.033297,0.000524,1.791162e-04,0.004090
2,0.002432,0.015599,0.000052,7.124623e-05,0.000426
3,0.021414,0.029498,0.000180,2.481610e-04,0.003050
4,0.001731,0.006889,0.000471,6.494214e-05,0.004840
...,...,...,...,...,...
91,0.003372,0.000636,0.001741,8.125273e-04,0.038397
92,0.000696,0.000377,0.000105,1.280831e-04,0.065695
93,0.002101,0.000174,0.000287,1.160753e-04,0.008619
94,0.001451,0.002320,0.000324,2.231448e-16,0.010899


In [ ]:
n_samples = 10
df_sparse = pd.DataFrame()
for i in range(n_samples):
    # TODO: find what distribution of signatures to use (Article uses 5 out of 10 for each sample)
    # Right now, use 0.4 percent chance of signatures being present, with the strength of it being between 0.5 and 2

    # Get a distribution of the counts
    distribution = [random.random()*1.5+0.5 if random.uniform(0, 1) > 0.6 else 0 for x in range(0, df_grch37.shape[1])]
    total = sum(distribution)
    while(sum(distribution)==0):
        distribution = [random.random()*1.5+0.5 if random.uniform(0, 1) > 0.6 else 0 for x in range(0, df_grch37.shape[1])]
        total = sum(distribution)

    # TODO: add Gaussian noise

    # normalize
    total = sum(distribution)
    distribution = [x/total for x in distribution]
    df_sparse[i] = distribution

df_sparse = df_sparse.set_index(df_grch37.columns)
df_sparse

,0,1,2,3,4,5,6,7,8,9
SBS18_GRCh37,0.000000,0.338219,0.462482,0.138475,0.525662,0.000000,0.000000,0.220589,0.214462,0.000000
SBS4_GRCh37,0.661623,0.396658,0.000000,0.269881,0.000000,0.349839,0.000000,0.000000,0.323544,0.000000
SBS6_GRCh37,0.338377,0.000000,0.000000,0.171527,0.000000,0.000000,0.273887,0.000000,0.461994,0.704707
SBS7a_GRCh37,0.000000,0.000000,0.000000,0.420117,0.309893,0.000000,0.000000,0.388867,0.000000,0.295293
SBS9_GRCh37,0.000000,0.265123,0.537518,0.000000,0.164445,0.650161,0.726113,0.390544,0.000000,0.000000


In [ ]:
simulated_data = df_grch37.dot(df_sparse)
for i in range(simulated_data.shape[1]):
    distribution = simulated_data[i]
    # Get the number of counts between 1001 and 51119 in logscale (51119 for easier numbers in formula)
    n_counts = 1000 + 10 ** (random.uniform(0, 4.7))
    counts = [int(x*n_counts) for x in distribution]
    simulated_data[i] = counts
    # print(n_counts)
    # print(max(counts))
    # print(sum(counts))
    # print('------')
simulated_data

,0,1,2,3,4,5,6,7,8,9
0,28,58,24,19,996,187,0,329,26,0
1,22,33,9,11,330,177,3,146,15,0
2,10,12,1,4,50,71,0,20,5,0
3,19,33,11,11,433,152,2,170,14,0
4,4,7,3,2,63,69,3,65,2,0
...,...,...,...,...,...,...,...,...,...,...
91,1,19,22,1,305,312,28,455,1,2
92,0,30,36,0,410,532,47,733,0,0
93,0,5,5,0,93,70,6,109,0,0
94,1,7,6,0,93,98,8,129,1,0


only simulate 30 samples (since real-life databases are also small (not in our case though??))

1. Choose 5 random signatures per sample to get a representation in combined probability (not really anymore since not sum to 100%
2. Add Gausian noise to the probabilites (or to the counts??)
3. randomly select between 1000 to 50000 mutations in log scale per sample


Still need to add Gaussian noise